# Steam API Caller
A collection of functions and calls for the steam API. Credentials and steam_ids are stored in a config file in a separate folder. I used my own and some friends' accounts for validation, but have only included demo steam IDs from the official steam documentation here. These are not real players, so some functions will return a null value (Recently Played Games and Shared Games). Some methods may be blocked if the user has their profile set to private rather than public. 

## Retrieve Config Details

In [1]:
import os
import configparser
import requests
import json
import pandas as pd
from IPython.display import HTML
import numpy as np

app_id = 440
dir_in = "//Users//haleyspeed//Docs//insight//api"
f_config = 'config.ini'
config = configparser.ConfigParser()
os.chdir(dir_in)
config.read(f_config)
steam_key = config.get('KEYS', 'steam')
steam_id = config.get('ID', 'demo_steam_id')
steam_id2 = config.get('ID', 'demo_steam_id3')


## Function Definitions

In [2]:
def unpack_json (txt):
    """Takes the output of an api call and unpacks it for """
    unpacked = json.loads(txt)
    return unpacked


## Modules (API calls + Formatting)


#### To get steam news per game

In [3]:
import datetime

def get_steam_game_news (app_id):
    url = 'http://api.steampowered.com/ISteamNews/GetNewsForApp/v0002/?appid=' + str(app_id) + '&format=json'
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_game_news (unpacked):
    df = pd.DataFrame ()
    for keys,appnews in unpacked.items():
        for keys,news_items in appnews.items():
            if keys == 'appid':
                row = [keys]
            if keys == 'newsitems':  
                for item in news_items:
                    df = df.append(item, ignore_index = True)
    #display(HTML(df.head().to_html()))  # For testing. Outputs a huge table
    return df

def display_news (news):
    for i,title in enumerate(news.title):
        display(HTML('<h1>' + title + '</h1>' +
                     '<br><strong>Author: </strong>' + news.author[i] + '<br>' +
                     '<strong>App ID: </strong>' + str(news.appid[i]) + '<br>' +
                     '<strong>Date: </strong>' + str(datetime.datetime.utcfromtimestamp(news.date[i], datetime.datetime.utc)) + '<br>' +
                     '<strong>ID: </strong>' + str(news.gid[i]) + '<br>' +
                     '<strong>URL: </strong>' + str(news.url[i]) + '<br>' + 
                     '<strong>Feed Label: </strong>' + str(news.feedlabel[i]) + '<br>' +
                     '<strong>Feed Name: </strong>' + str(news.feedname[i]) + '<br>' +
                     '<strong>Feed Type: </strong>' + str(news.feed_type[i]) + '<br>'))
        display(HTML('<br>' + news.contents[i].replace('[','<').replace(']','>').replace('<*>','<br>* ').replace('<h1>','<h3>').replace('</h1>','</h3>') + '<hr><br>'))
                 


game_news = get_steam_game_news (app_id)
unpacked = unpack_json (game_news)
df = explore_game_news(unpacked)
display_news (df[:2])

#### To get global achievement stats per game

In [4]:
def get_steam_global_achievement_progress (app_id):
    url = 'http://api.steampowered.com/ISteamUserStats/GetGlobalAchievementPercentagesForApp/v0002/?gameid=' + str(app_id) + '&format=json'
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_global_achievements (unpacked):
    df = pd.DataFrame (columns = ['achievement', 'percent'])
    name = ''
    percent = np.nan
    for keys,achievementpercentages in unpacked.items():
        for keys, achievements in achievementpercentages.items():
            for i,achievement in enumerate(achievements):
                for keys,names_percent in achievement.items():
                    if isinstance(names_percent, str)==True:
                        name = names_percent
                    if isinstance(names_percent, float) == True or isinstance(names_percent, int):
                        percent = names_percent
                        df = df.append({"achievement":name, "percent":percent}, ignore_index = True) 
                        
    display(HTML(df.head().to_html()))
    return df

global_achieves = get_steam_global_achievement_progress(app_id)
unpacked = unpack_json (global_achieves)
df = explore_global_achievements(unpacked)

,achievement,percent
0,TF_SCOUT_LONG_DISTANCE_RUNNER,55.200001
1,TF_HEAVY_DAMAGE_TAKEN,45.799999
2,TF_GET_CONSECUTIVEKILLS_NODEATHS,42.299999
3,TF_PYRO_CAMP_POSITION,34.799999
4,TF_KILL_NEMESIS,32.400002


#### Get Steam Global Game Stats

In [5]:
def get_steam_global_stats (app_id):
    url = 'http://api.steampowered.com/ISteamUserStats/GetGlobalStatsForGame/v0001/?format=json&appid=' + str(app_id) + '&count=1&name%5B0%5D=global.map.emp_isle'
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_steam_global_stats (unpacked):
    df = pd.DataFrame ()
    for keys,response in unpacked.items():
        for keys, values in response.items():
            if isinstance(values,dict):
                for keys, results in values.items():
                    tmp = pd.DataFrame()
                    for keys, more_results in results.items():
                        tmp[keys] = [more_results]
                    df = df.append(tmp)
            else:
                df[keys] = [values]
                    
    display(HTML(df.head().to_html()))
    return df
            

app_id = 22380 # Empires Mod
game_stats = get_steam_global_stats (app_id)
unpacked = unpack_json (game_stats)
df = explore_steam_global_stats (unpacked)

,result,error
0,8,Stat 'global.map.emp_isle' not found


#### Get Player Summary

In [6]:
def get_player_summaries (steam_key, steam_id):
    url = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=' + str(steam_key) + '&steamids=' + str(steam_id)
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_player_summaries (unpacked):
    df = pd.DataFrame ()
    for keys,response in unpacked.items():
        for keys,players in response.items():
            for player in players:
                tmp = pd.DataFrame()
                for keys,values in player.items():
                    tmp[keys] = [values]
                df = df.append(tmp)
    df = df.transpose()
    display(HTML(df.head().to_html()))
    return df

player_summary = get_player_summaries (steam_key, steam_id2)
unpacked = unpack_json (player_summary)
df = explore_player_summaries (unpacked)

,0
steamid,76561197960435530
communityvisibilitystate,3
profilestate,1
personaname,Robin
profileurl,https://steamcommunity.com/id/robinwalker/


#### Get Player's Friends List

In [7]:
import datetime

def get_player_friends (steam_key, steam_id):
    url = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key=' + str(steam_key) + '&steamid=' + str(steam_id) + '&relationship=friend'
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_player_friends (unpacked):
    df = pd.DataFrame ()
    for keys, friendslist in unpacked.items():
        for keys, friends in friendslist.items():
            for friend in friends:
                tmp = pd.DataFrame()
                for keys, values in friend.items():
                    if "friend_since" in keys:
                        values = datetime.datetime.utcfromtimestamp(values, datetime.datetime.utc)
                    tmp[keys] = [values]
                df = df.append(tmp)   
    display(HTML(df.head().to_html()))
    return df

player_friends = get_player_friends (steam_key, steam_id2)
unpacked = unpack_json (player_friends)
df = explore_player_friends (unpacked)

,steamid,relationship,friend_since
0,76561197960265731,friend,1970-01-01 00:00:00
0,76561197960265738,friend,1970-01-01 00:00:00
0,76561197960265740,friend,1970-01-01 00:00:00
0,76561197960265744,friend,2020-03-29 19:03:33
0,76561197960265747,friend,1970-01-01 00:00:00


#### Get Player Achievements

In [8]:
def get_player_achievements (app_id,steam_key, steam_id):
    url = 'http://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid=' + str(app_id) + '&key=' + str(steam_key) + '&steamid=' + str(steam_id)
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_player_achievements(unpack):
    df = pd.DataFrame ()
    for keys, playerstats in unpack.items():
        for keys,steam_id in playerstats.items():
            if 'error' in keys:
                print('Profile is not public')
            else:
                if "achievements" in keys:
                    for achievements in steam_id:
                        tmp = pd.DataFrame()
                        for keys, values in achievements.items():
                            if 'unlocktime' in keys:
                                values = datetime.datetime.utcfromtimestamp(values,datetime.datetime.utc)
                            tmp[keys] = [values]
                        df = df.append(tmp)
    display(HTML(df.head().to_html()))
    return df

player_achievements = get_player_achievements (app_id,steam_key, steam_id)
unpacked = unpack_json(player_achievements)
df = explore_player_achievements (unpacked)

Profile is not public


""


#### Get Player Stats Per Game

In [9]:
## Gives a server Internal Error even with example URL
def get_user_stats_per_game (app_id,steam_key,steam_id):
    url = 'http://api.steampowered.com/ISteamUserStats/GetUserStatsForGame/v0002/?appid='+ str(app_id) + '&key=' + str(steam_key) + '&steamid=' + str(steam_id)
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked


player_stats = get_user_stats_per_game (app_id,steam_key,steam_id)
print(player_stats)

<html>
<head>
<title>500 Internal Server Error</title>
</head>
<body>
<h1>Internal Server Error</h1>
</body>
</html>


#### Get Recently Played Games

In [10]:
def get_recently_played_games (steam_key,steam_id):
    url = 'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key=' + str(steam_key) + '&steamid=' + str(steam_id) + '&format=json'
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_recent_games (unpacked):
    df = pd.DataFrame ()
    for keys,values in unpacked.items():
        for key, games in values.items():
            if 'games' in key:
                for game in games:
                    tmp = pd.DataFrame()
                    for key, item in game.items():
                        tmp [key] = [item]
                    df = df.append(tmp)
    display(HTML(df.head().to_html()))
    return df

recent_games = get_recently_played_games (steam_key,steam_id2)
unpacked = unpack_json (recent_games)
df = explore_recent_games (unpacked)

""


#### Get Shared Games

In [11]:
def get_shared_games (steam_key,steam_id):
    url = 'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key=' + str(steam_key) + '&steamid=' + str(steam_id) + '&format=json'
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_shared_games (unpacked):
    df = pd.DataFrame ()
    for keys,values in unpacked.items():
        for keys, total_count in values.items():
            if isinstance(total_count, list):
                tmp = pd.DataFrame()
                for count in total_count:
                    for key, value in count.items():
                        tmp[key] = [value]
                    df = df.append(tmp)
    display(HTML(df.head().to_html()))
    return df

shared_games = get_shared_games (steam_key,steam_id)
unpacked = unpack_json (shared_games)
df = explore_shared_games (unpacked)

""


#### Get Game Schema

In [12]:
def get_game_schema (app_id, steam_key):
    url = 'http://api.steampowered.com/ISteamUserStats/GetSchemaForGame/v2/?key=' + str(steam_key)+ '&appid= '+ str(app_id)
    r = requests.get(url, data = {'key':'value'})
    return r.text

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked

def explore_game_schema (unpacked):
    df = pd.DataFrame ()
    gameName = ''
    gameVersion = np.nan
    for keys,game in unpacked.items():
        i = 0
        for field, subfield in game.items():
            if field == 'gameName':
                gameName = subfield
            elif field == 'gameVersion':
                gameVersion = subfield
            elif field == 'availableGameStats':
                tmp = pd.DataFrame()
                for keys,values in subfield.items():
                    for value in values:
                        for keys, schema in value.items():
                            tmp[keys] = [schema]
                            tmp['gameName'] = gameName
                            tmp['gameVersion'] = gameVersion
                        df = df.append(tmp)
    display(HTML(df.head().to_html()))
    return df

game_schema = get_game_schema (app_id, steam_key)
unpacked = unpack_json(game_schema)
df = explore_game_schema (unpacked)

,name,gameName,gameVersion,defaultvalue,displayName,hidden,description,icon,icongray
0,A01,ValveTestApp22380,25,0,New Kid,0,Reached 10th level.,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/57ffeb21c637010758bf87f708a47ff8c62256ef.jpg,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/6c7f9d60669a89566e3aa1b19a460574407540d8.jpg
0,A02,ValveTestApp22380,25,0,Up and Comer,0,Reach 20th level.,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/3cc42e549932da26fcbb049bd46f35893b98e78c.jpg,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/281021c27b423faa73c87762ce6ffb2823d78a17.jpg
0,A03,ValveTestApp22380,25,0,The Boss,0,Reach 30th level.,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/e2395a7100759c3e1d422bfe98dbe860941329a3.jpg,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/a9ce0a7b692913f0f1f0b4cdd7fda22ee61a8332.jpg
0,A04,ValveTestApp22380,25,0,Ol' Buddy Ol' Pal,0,Recruit any companion.,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/5f627959d8b341f1e0e4268e21448bb9b7986132.jpg,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/87437a68f94c051099420c587d2badef41e6e102.jpg
0,A05,ValveTestApp22380,25,0,The Whole Gang's Here,0,Recruit all companions.,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/6ec46779e08726ef3c936a1bde010e66abea7eac.jpg,https://steamcdn-a.akamaihd.net/steamcommunity/public/images/apps/22380/ab634a9cdf83baedd628bc4dd54e65e43dc6f46e.jpg


#### Get Player Bans

In [13]:
def get_player_bans (app_id, steam_key):
    url = 'http://api.steampowered.com/ISteamUser/GetPlayerBans/v1/?key=' + str(steam_key)+ '&steamids='+ str(steam_id)
    r = requests.get(url, data = {'key':'value'})
    return r.text


def explore_player_bans (unpacked):
    df = pd.DataFrame ()
    for keys,values in unpacked.items():
        tmp = pd.DataFrame()
        for key, value in values[0].items():
            tmp[key] = [value]
        df = df.append(tmp)
    display(HTML(df.head().to_html()))
    return df

def unpack_json (txt):
    unpacked = json.loads(txt)
    return unpacked


player_bans = get_player_bans (app_id, steam_key)
unpacked = unpack_json (player_bans)
df = explore_player_bans(unpacked)

,SteamId,CommunityBanned,VACBanned,NumberOfVACBans,DaysSinceLastBan,NumberOfGameBans,EconomyBan
0,76561197972495328,False,False,0,0,0,none
